In [325]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
import awkward as ak
from plotly import graph_objects as go, express as px
import plotly.io as pio
pio.templates['plotly'].layout['autosize'] = False # <---This did the trick.


In [331]:
hist_name = "mu_pi_hnl.root"
# Open the ROOT file
root_file = uproot.open(hist_name)
path = 'generator/mevprtl_gen/'
df = root_file[path].arrays(library="pd")

In [332]:
df.keys()

Index(['meson_dmom', 'meson_dmom_beamcoord', 'meson_dpos_beamcoord',
       'meson_pdg', 'mevprtl_mom_beamcoord', 'mevprtl_mom', 'mevprtl_start',
       'equiv_enu', 'mevprtl_enter', 'mevprtl_exit', 'decay_pos',
       'daughter_mom', 'daughter_e', 'daughter_pdg', 'pot', 'flux_weight',
       'ray_weight', 'decay_weight', 'mass', 'C1', 'C2', 'C3', 'C4', 'C5',
       'total_decay_width', 'total_mean_lifetime', 'total_mean_distance',
       'allowed_decay_fraction', 'gen'],
      dtype='object')

In [346]:
df['mevprtl_mom'][0]['fP']

<Record {fX: -0.000139, fY: 0.0219, ...} type='TVector3[fX: float64, fY: fl...'>

In [351]:
def convert_beamcoords(vec):

    sbnd_bnb_beam_origin = [-73.78, 0, -11000] # cm - beam in detector frame
    return vec + np.array(sbnd_bnb_beam_origin)

In [363]:
pdg_color_name_dict = {
    11: ['blue',  'e-'],
    12: ['blue',  'nu_e'],
    13: ['red',   'mu-'],
    14: ['red',   'nu_mu'],
    211: ['green', 'pi+'],
    111: ['green', 'pi0'],
    321: ['orange', 'K+'],
    311: ['yellow', 'K0L'],
    221: ['purple', 'eta'],
    22: ['black', 'gamma'],
}

sbnd_det_box = {
    'x': np.array([-200,200]),
    'y': np.array([-200,200]),
    'z': np.array([0, 500])
}

In [411]:
def get_parent_decay_point(df):
    dpos_key = 'meson_dpos_beamcoord'
    d_pos_arr = []
    df[dpos_key].apply(lambda x: d_pos_arr.append([x['fP']['fX'], x['fP']['fY'], x['fP']['fZ'], x['fE']]))
    d_pos_arr = np.array(d_pos_arr)
    d_pos_arr[:, :3] = convert_beamcoords(d_pos_arr[:, :3])
    df['parent_dpos'] = list(d_pos_arr[:, :3])
    df['parent_dT'] = d_pos_arr[:, 3]
    return df

def get_mevprtl_decay_point(df):
    dpos_key = 'decay_pos'
    d_pos_arr = []
    df[dpos_key].apply(lambda x: d_pos_arr.append([x['fP']['fX'], x['fP']['fY'], x['fP']['fZ'], x['fE']]))
    d_pos_arr = np.array(d_pos_arr)
    df['mevprtl_dpos'] = list(d_pos_arr[:, :3])
    df['mevprtl_dT'] = d_pos_arr[:, 3]

    return df

def get_mevprtl_mom(df):
    # Convert to awkward array for vectorized operations
    mevprtl_mom_ak = ak.Array(df['mevprtl_mom'])
    
    # Extract momentum components vectorized
    mass = df['mass']
    px = mevprtl_mom_ak['fP']['fX']
    py = mevprtl_mom_ak['fP']['fY'] 
    pz = mevprtl_mom_ak['fP']['fZ']
    E = np.sqrt(px**2 + py**2 + pz**2 + mass**2)  # Calculate energy from momentum and mass
    # Add momentum components as new columns
    df['mevprtl_px'] = px.to_list()
    df['mevprtl_py'] = py.to_list()
    df['mevprtl_pz'] = pz.to_list()
    df['mevprtl_E'] = E.to_list()
    return df

def get_child_mom(df):
    """Extract daughter momentum using vectorized awkward array operations"""
    # Convert to awkward array for vectorized operations
    daughter_mom_ak = ak.Array(df['daughter_mom'])
    
    # Extract momentum components vectorized
    px = daughter_mom_ak['fX']
    py = daughter_mom_ak['fY'] 
    pz = daughter_mom_ak['fZ']
    
    # Add momentum components as new columns
    df['daughter_px'] = daughter_mom_ak['fX'].to_list()
    df['daughter_py'] = daughter_mom_ak['fY'].to_list()
    df['daughter_pz'] = daughter_mom_ak['fZ'].to_list()
    return df

def make_detector_box(fig, xlims=sbnd_det_box['x'], ylims=sbnd_det_box['y'], zlims=sbnd_det_box['z']):
    
    fig.add_trace(
        go.Scatter3d(
            x=[xlims[0], xlims[1], xlims[1], xlims[0], xlims[0],xlims[0], xlims[1], xlims[1], xlims[0], xlims[0], xlims[0], xlims[0], xlims[1], xlims[1], xlims[1], xlims[1], 0, 0, 0, 0, 0],
            y=[zlims[0], zlims[0], zlims[1], zlims[1], zlims[0], zlims[0], zlims[0], zlims[1], zlims[1], zlims[0], zlims[1], zlims[1], zlims[1], zlims[1],zlims[0], zlims[0], zlims[0], zlims[0], zlims[1], zlims[1], zlims[0]],
            z=[ylims[0], ylims[0], ylims[0], ylims[0], ylims[0], ylims[1], ylims[1], ylims[1], ylims[1], ylims[1], ylims[1], ylims[0], ylims[0], ylims[1], ylims[1], ylims[0],ylims[0], ylims[1], ylims[1], ylims[0], ylims[0]],
            mode='lines',
            line=dict(color='lightgrey', width=2),
            name='Detector Box'
        )
    )
    return fig

def plot_3d(df, event_list=None, vector_scale=1000):
    if event_list is None:
        event_list = df_temp.index.tolist()
    elif type(event_list) is list:
        pass
    elif type(event_list) is not list and type(event_list) is int:
        event_list = [event_list]
    else:
        raise ValueError("event_list must be a list, integer or None")
    
    df_temp = get_parent_decay_point(df.copy())
    df_temp = get_mevprtl_decay_point(df_temp)
    df_temp = get_child_mom(df_temp)
    df_temp = get_mevprtl_mom(df_temp)

    
    for event in event_list:
        parent_coords = df_temp['parent_dpos'][event]
        mevprtl_coords = df_temp['mevprtl_dpos'][event]

        # Create a 3D scatter plot

        fig = go.Figure(data=[go.Scatter3d(
            x=[mevprtl_coords[0]],
            y=[mevprtl_coords[2]],
            z=[mevprtl_coords[1]],
            mode='markers',
            marker=dict(size=6, color='blue',symbol='cross'),
            name='MeVPRTL Decay Point'
        )])

        fig.add_trace(go.Scatter3d(
            x=[parent_coords[0]],
            y=[parent_coords[2]],
            z=[parent_coords[1]],
            mode='markers',
            marker=dict(size=5, color='red'),
            name='Parent Decay Point'
        ))

        decay_vector = np.array(mevprtl_coords) - np.array(parent_coords)
        len_decay = np.linalg.norm(decay_vector)
        decay_norm = decay_vector / len_decay
        
        mevprtl_E = df_temp['mevprtl_E'][event]

        decay_norm *= vector_scale * mevprtl_E  # Scale the decay vector by energy for visibility

        fig.add_trace(go.Scatter3d(
            x=[mevprtl_coords[0]-decay_norm[0], mevprtl_coords[0]],
            y=[mevprtl_coords[2]-decay_norm[2], mevprtl_coords[2]],
            z=[mevprtl_coords[1]-decay_norm[1], mevprtl_coords[1]],
            mode='lines',
            line=dict(color='black', dash='dot', width=4),
            name='Decay Path'
        ))

        nchildren = len(df_temp['daughter_pdg'][event])

        for child in range(nchildren):
            pdg = df_temp['daughter_pdg'][event][child]
            if np.abs(pdg) in pdg_color_name_dict:
                daughter_color, name = pdg_color_name_dict[np.abs(pdg)]
            else:
                daughter_color = 'grey'
                name = f'Unknown ({pdg})'
            child_vec = np.array([df_temp['daughter_px'][event][child], df_temp['daughter_py'][event][child], df_temp['daughter_pz'][event][child]])
            
            
            vec_len = np.linalg.norm(child_vec)
            child_vec = child_vec / vec_len  # Scale the vector for visibility
            
            child_energy = df_temp['daughter_e'][event][child]
            child_vec *= vector_scale*child_energy  # Scale the vector by energy for visibility
            
            if 'nu' in name:
                fig.add_trace(go.Scatter3d(
                    x=[mevprtl_coords[0],mevprtl_coords[0]+child_vec[0]],
                    y=[mevprtl_coords[2],mevprtl_coords[2]+child_vec[2]],
                    z=[mevprtl_coords[1],mevprtl_coords[1]+child_vec[1]],
                    mode='lines',
                    line=dict(color=daughter_color, width=4, dash='dash'),
                    name=name
                ))
            else:
                fig.add_trace(go.Scatter3d(
                    x=[mevprtl_coords[0],mevprtl_coords[0]+child_vec[0]],
                    y=[mevprtl_coords[2],mevprtl_coords[2]+child_vec[2]],
                    z=[mevprtl_coords[1],mevprtl_coords[1]+child_vec[1]],
                    mode='lines',
                    line=dict(color=daughter_color, width=4),
                    name=name
                ))



        fig = make_detector_box(fig)

        
        
        width_x = sbnd_det_box['x'][1] - sbnd_det_box['x'][0]
        width_y = sbnd_det_box['y'][1] - sbnd_det_box['y'][0]
        width_z = sbnd_det_box['z'][1] - sbnd_det_box['z'][0]

        x_lims = [sbnd_det_box['x'][0]-0.1*width_x, sbnd_det_box['x'][1]+0.1*width_x]
        y_lims = [sbnd_det_box['y'][0]-0.1*width_y, sbnd_det_box['y'][1]+0.1*width_y]
        z_lims = [sbnd_det_box['z'][0]-0.1*width_z, sbnd_det_box['z'][1]+0.1*width_z]
        
        fig.update_scenes(xaxis_title_text='X (cm)',  
                  yaxis_title_text='Z (cm)',  
                  zaxis_title_text='Y (cm)',
                  xaxis=dict(range=x_lims),
                  yaxis=dict(range=z_lims),
                  zaxis=dict(range=y_lims),
                  aspectmode='manual',
                  aspectratio=dict(x=1, y=1, z=1)
        ),
        camera = dict(
            up=dict(x=0, y=0, z=1),
            center=dict(x=0, y=0, z=0),
            eye=dict(x=1.25, y=-1.25, z=1.25)
        )

        fig.update_layout(scene_camera=camera)
        fig.layout.height = 800
        fig.layout.width = 1200
    
        fig.show()

def plot_decay_path(df, event_list=None):
    if event_list is None:
        event_list = df_temp.index.tolist()
    elif type(event_list) is list:
        pass
    elif type(event_list) is not list and type(event_list) is int:
        event_list = [event_list]
    else:
        raise ValueError("event_list must be a list, integer or None")

    df_temp = get_parent_decay_point(df.copy())
    df_temp = get_mevprtl_decay_point(df_temp)

    fig = go.Figure()
    fig = make_detector_box(fig)
    for event in event_list:
        parent_coords = df_temp['parent_dpos'][event]
        mevprtl_coords = df_temp['mevprtl_dpos'][event]

        fig.add_trace(go.Scatter3d(
            x=[parent_coords[0], mevprtl_coords[0]],
            y=[parent_coords[2], mevprtl_coords[2]],
            z=[parent_coords[1], mevprtl_coords[1]],
            mode='lines',
            line=dict(dash='dot'),
            name='Event: {}'.format(event),
        ))

    camera = dict(
        up=dict(x=0, y=0, z=1),
        center=dict(x=0, y=0, z=0),
        eye=dict(x=1.25, y=-1.25, z=1.25)
    )

    fig.update_layout(scene_camera=camera)
    fig.update_scenes(
                aspectmode='manual',
                aspectratio=dict(x=1, y=5, z=1),
                xaxis_title_text='X (cm)',  
                yaxis_title_text='Z (cm)',  
                zaxis_title_text='Y (cm)',)
    fig.layout.height = 800
    fig.layout.width = 1200


    fig.show()

In [412]:
events_to_plot = [3]
plot_decay_path(df, event_list=events_to_plot)
plot_3d(df, event_list=events_to_plot, vector_scale=500)